# Sistema de aquisição de dados de voz normal e sussurrada

Criado por: Cezar Yamamura, Rogério Pignelli, Prof. Dr. Paulo Scalassara

Data: 19 set 2024

### 1 - Definir o código do voluntário

In [1]:
speaker_code = "speakerM01"

### 2 - Carregar as funções e as bibliotecas

In [19]:
import sounddevice as sd
import soundfile as sf
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import numpy as np
import sys
import matplotlib.pyplot as plt
import IPython


# Configurações de gravação
sample_rate = 44100  # 44.1kHz
channels = 1
recording = None  # Variável global para armazenar o áudio gravado
mode = 'normal'  # Modo de gravação: 'normal' ou 'sussurrada'
is_recording = False
recording = []
current_index = 0 

# Função para carregar as palavras do arquivo tasks1.txt
def load_words_from_file(filepath):
    try:
        with open(filepath, 'r') as file:
            file_words = [line.strip() for line in file.readlines()]
        return file_words
    except FileNotFoundError:
        print(f"Arquivo {filepath} não encontrado.")
        return []
    
# Função de callback para capturar o áudio continuamente
def callback(indata, frames, time, status):
    if is_recording:
        recording.append(indata.copy())

# Função para iniciar a gravação
def start_recording(b):
    global recording, is_recording, stream
    clear_output()  # Limpa a saída do notebook
    recording = []  # Reseta a gravação anterior
    is_recording = True
    print(f"Iniciando a gravação do task 't{current_index + task_aux + 1}' da palavra '{words[current_index]}' com fala {mode} ")
    display(stop_button)
    # Inicia o fluxo de entrada com callback
    stream = sd.InputStream(samplerate=sample_rate, channels=channels, callback=callback)
    stream.start()  

# Função para parar a gravação e salvar o arquivo
def stop_recording(b):
    global recording, current_index, mode, stream, is_recording, speaker_code
    is_recording = False
    stream.stop()
    stream.close()
    clear_output()  # Limpa a saída do notebook
    print("Gravação parada.")
    
    # Concatena todos os segmentos gravados
    audio_data = np.concatenate(recording, axis=0)
    
    # Definir código do speaker e task
    
    task_number = f"t{current_index + task_aux + 1:02d}"  # Task com zero à esquerda, ex: t01, t02...
    
    # Define modo de fala
    if mode == 'normal':
        mode_code = "N"
    else:
        mode_code = "W"

    # Definir nome do arquivo no formato desejado
    filename = f"{speaker_code}_{task_number}_{mode_code}.wav"
    
    # Cria a pasta se não existir
    folder = speaker_code
    if not os.path.exists(folder):
        os.makedirs(folder)

    # Caminho completo do arquivo
    filepath = os.path.join(folder, filename)

    # Calcula o número de amostras a serem cortadas (100 ms) Por causa do click do mouse
    sample_rate = 44100
    cut_samples = int(sample_rate * 0.25)  # 10 ms

    # Corta os primeiros e últimos 10 ms
    if len(audio_data) > 2 * cut_samples:
        audio_data = audio_data[cut_samples:-cut_samples]
    else:
        print("Áudio muito curto para aplicar o corte.")

    # Salva o arquivo de áudio
    sf.write(filepath, audio_data, sample_rate)
    print(f"Gravação salva como {filepath}")

    display(next_button, restart_button)

    # Calcula o tempo em segundos
    time = np.arange(len(audio_data)) / sample_rate
    plt.figure(figsize=(10, 4))
    plt.plot(time,audio_data)
    plt.title(f"Waveform - {filename}")
    plt.xlabel("Amostras")
    plt.ylabel("Amplitude")
    plt.grid()
    plt.show()

    # Criar o widget de áudio para reprodução
    display(IPython.display.Audio(filename=filepath))

# Função para avançar para a próxima palavra
def next_word(b):
    global current_index, mode
    clear_output()  # Limpa a saída do notebook

    # Alterna entre fala normal e sussurrada
    if mode == 'normal':
        mode = 'sussurrada'
    else:
        mode = 'normal'
        current_index += 1  # Passa para a próxima palavra
        

    if current_index < len(words):
        print(f"Grave a palavra '{words[current_index]}' do task 't{current_index + task_aux + 1:02d}' com fala {mode}:")
        display(start_button, stop_button, next_button)
    else:
        print("Todas as palavras foram gravadas.")
        current_index = 0  # Reinicia a contagem se necessário

 # Função para iniciar o sistema
def start_system(b):
    global current_index, mode
    clear_output()  # Limpa a saída do notebook

    print(f"Grave a palavra '{words[current_index]}' do task 't{current_index + task_aux + 1:02d}' com fala {mode}:")
    display(start_button, stop_button, next_button)


### 3 - Iniciar o Task 1: Fonemas Sustentados 

O voluntário pronunciará cada som três vezes em fala normal e três vezes em sussurro, com duração de 1 a 3 segundos cada. 

In [23]:
# Carrega as palavras do arquivo e filtra a lista original
words = load_words_from_file('tasks/tasks1.txt')
task_aux = 0

start_button = widgets.Button(description="Iniciar Gravação")
stop_button = widgets.Button(description="Parar Gravação")
next_button = widgets.Button(description="Próximo")
start_system_button = widgets.Button(description="Start System")
restart_button = widgets.Button(description="Gravar Novamente")

# Atribui funções aos botões
start_button.on_click(start_recording)
stop_button.on_click(stop_recording)
next_button.on_click(next_word)
start_system_button.on_click(start_system)
restart_button.on_click(start_recording)

# Inicia a interface
print("Clique em 'Start System' para iniciar a gravação:")
display(start_system_button)

Clique em 'Start System' para iniciar a gravação:


Button(description='Start System', style=ButtonStyle())

### 4 - Iniciar o Task 2: Leitura de Palavras 

O voluntário pronunciará cada palavra três vezes em fala normal e três vezes em sussurro. 

In [25]:
# Carrega as palavras do arquivo e filtra a lista original
words = load_words_from_file('tasks/tasks2.txt')
task_aux = 13

start_button = widgets.Button(description="Iniciar Gravação")
stop_button = widgets.Button(description="Parar Gravação")
next_button = widgets.Button(description="Próximo")
start_system_button = widgets.Button(description="Start System")
restart_button = widgets.Button(description="Gravar Novamente")

# Atribui funções aos botões
start_button.on_click(start_recording)
stop_button.on_click(stop_recording)
next_button.on_click(next_word)
start_system_button.on_click(start_system)
restart_button.on_click(start_recording)

# Inicia a interface
print("Clique em 'Start System' para iniciar a gravação:")
display(start_system_button)

Clique em 'Start System' para iniciar a gravação:


Button(description='Start System', style=ButtonStyle())

### 5 - Iniciar o Task 3: Leitura de Frases

O voluntário lerá frases três vezes em fala normal e três vezes em sussurro.

In [26]:
# Carrega as palavras do arquivo e filtra a lista original
words = load_words_from_file('tasks/tasks3.txt')
task_aux = 38

start_button = widgets.Button(description="Iniciar Gravação")
stop_button = widgets.Button(description="Parar Gravação")
next_button = widgets.Button(description="Próximo")
start_system_button = widgets.Button(description="Start System")
restart_button = widgets.Button(description="Gravar Novamente")

# Atribui funções aos botões
start_button.on_click(start_recording)
stop_button.on_click(stop_recording)
next_button.on_click(next_word)
start_system_button.on_click(start_system)
restart_button.on_click(start_recording)

# Inicia a interface
print("Clique em 'Start System' para iniciar a gravação:")
display(start_system_button)

Clique em 'Start System' para iniciar a gravação:


Button(description='Start System', style=ButtonStyle())

### 6 - Iniciar o Task 4: Leitura de texto

O voluntário lerá 1 texto uma vez em fala normal e uma vez em sussurro. 

O vento norte e o sol discutiam qual dos dois era o mais forte, quando sucedeu passar um viajante envolto numa capa. Ao vê-lo, põem-se de acordo em como aquele que primeiro conseguisse obrigar o viajante a tirar a capa seria considerado o mais forte. O vento norte começou a soprar com muita fúria, mas quanto mais soprava, mais o viajante se aconchegava à sua capa, até que o vento norte desistiu. O sol brilhou então com todo o esplendor, e imediatamente o viajante tirou a capa. O vento norte teve assim de reconhecer a superioridade do sol.

In [27]:
# Carrega as palavras do arquivo e filtra a lista original
words = load_words_from_file('tasks/tasks4.txt')
task_aux = 48

start_button = widgets.Button(description="Iniciar Gravação")
stop_button = widgets.Button(description="Parar Gravação")
next_button = widgets.Button(description="Próximo")
start_system_button = widgets.Button(description="Start System")
restart_button = widgets.Button(description="Gravar Novamente")

# Atribui funções aos botões
start_button.on_click(start_recording)
stop_button.on_click(stop_recording)
next_button.on_click(next_word)
start_system_button.on_click(start_system)
restart_button.on_click(start_recording)

# Inicia a interface
print("Clique em 'Start System' para iniciar a gravação:")
display(start_system_button)

Clique em 'Start System' para iniciar a gravação:


Button(description='Start System', style=ButtonStyle())